In [ ]:
!pip install faiss-cpu
!pip install langchain
!pip install langchain-openai

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import faiss
import ast

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
db = FAISS.load_local("/content/drive/MyDrive/Address_embedded", embeddings,allow_dangerous_deserialization=True)

In [ ]:
query = "Give me details about the resident at Swarnadhama Nagar, Old Bowenpally, Telangana 500011."
docs = db.similarity_search(query)

In [ ]:
docs

[Document(page_content='Consider the following information for the address of KONDAM SRIDHAR REDDY:\n- House Number: 8-7-177/50/40,41 FLAT NO 201\n- Building Name: KAMALA RESIDENCY\n- Street Name: SWARNADHAMA NAGAR\n- Locality: OLD BOWENPALLY\n- Landmark: VENKATESHWARA TEMPLE\n- State: Telangana\n- District: Medchal-Malkajgiri\n- Pin Code: 500011'),
 Document(page_content='Consider the following information for the address of BONDUGULA  SUMAN REDDY:\n- House Number: 5-20/1\n- Building Name: SWATHI RESIDENCY\n- Street Name: ROAD NO 1, BUDDHA NAGAR COLONY\n- Locality: BODUPPAL\n- Landmark: NA\n- State: Telangana\n- District: Ranga Reddy\n- Pin Code: 500092'),
 Document(page_content='Consider the following information for the address of M/s K S R CONSTRUCTIONS:\n- Mandal: \n- Village/City/Town: OLD BOWENPALLY\n- Street: OLD BOWENPALLY VILLAGE\n- Locality: 500011\n- State: Medchal-Malkajgiri\n- District: Balanagar\n- Pin Code:'),
 Document(page_content='Consider the following information f

In [ ]:
print(docs[0].page_content)

Consider the following information for the address of KONDAM SRIDHAR REDDY:
- House Number: 8-7-177/50/40,41 FLAT NO 201
- Building Name: KAMALA RESIDENCY
- Street Name: SWARNADHAMA NAGAR
- Locality: OLD BOWENPALLY
- Landmark: VENKATESHWARA TEMPLE
- State: Telangana
- District: Medchal-Malkajgiri
- Pin Code: 500011


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load Flan model
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def query_template(information,query):
  prompt_db = f"""
  Query Analysis:

Query Analysis:

Query: {query}

{information}

Instructions: Generate the output based on the provided information.

Output:
"""
  return prompt_db

In [ ]:
dash_line = '-'.join('' for x in range(100))
def Q_A_address(query):
  entities=[]
  docs = db.similarity_search(query)
  entities.append(docs[0].page_content)

  str =""
  for i in entities:
    prompt = query_template(i,query)
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(dash_line)
    print("QUERY:",query)
    print('Similarity Search:-',i)
    print("RESPONSE:", response)
    # str =dash_line+"\n"+"QUERY:"+query+"\n"+'Similarity Search:'+i+"\n"+"RESPONSE:"+ response+"\n"
    str = response
  return str

In [ ]:
Q_A_address("Address of upala?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


---------------------------------------------------------------------------------------------------
QUERY: Address of upala?
Similarity Search:- Consider the following information for the address of Uppala  Srinivas:
- House Number: 2-3-906/1
- Building Name: Plot No 189
- Street Name: Road No 5
- Locality: Co Op Bank Colony
- Landmark: Shiva Function Hall
- State: Telangana
- District: Medchal-Malkajgiri
- Pin Code: 500068
RESPONSE: Uppala Srinivas is located at Co Op Bank Colony, Medchal-


'Uppala Srinivas is located at Co Op Bank Colony, Medchal-'

In [ ]:
old_queries_address = [
    "Can you provide the current address of Vamshi Constructions?",
    "In which state is the residence of Sri Satya Developers located?",
    "What is the postal code for Divinehomes's current residence?",
    "Could you please share the complete address details of Sri Vathsa Homes Pvt Ltd?",
    "Which state does Sri Sreenivasa Constructions currently reside in?",
    "What is the address of BSR Developers and Others?",
    "Do you have the postal code for Devi Developers & Builders's current residence?",
    "What is the full address of Ektha Prime Lakemont Homes?",
    "In which state does Ashoka Ventures Rep by M Ashok Vardhan currently reside?",
    "What is the postal code for Akshay Developers's current residence?",
    "Can you provide the address details for Agalduty Raju?",
    "In which state does AV Constructions and Developers currently reside?",
    "What is the postal code for Vathsalya Constructions's residence?",
    "Could you please share the complete address of Sreshta Constructions?",
    "Which state is the current residence of Sri Sai Chakra Developers located in?",
    "Do you have the postal code for Bricks Infratech LLP's residence?",
    "What is the address of Eminence Constructions?",
    "What is the postal code for Rudraksh MegaCorp Private Limited's current residence?",
    "Could you share the complete address of Sri Sai Ram Projects?",
    "In which state does AVL Constructions currently reside?",
    "What is the postal code for VB City LLP's residence?",
    "Can you provide the address details for Shashikiran Builders & Developers?",
    "What is the current address of Legado Ventures LLP?",
    "In which state does Goldstone Infra currently reside?",
    "What is the postal code for JNR Builders's residence?",
    "Could you please share the complete address of Dhanush Infrastructers?",
    "Which state does M/S Shiva Sharan Infra Developers currently reside in?",
    "Do you have the postal code for New Perfect Builders's residence?",
    "What is the address of My Place Infra?",
    "What is the postal code for Swetcha Destinations LLP's current residence?",
    "Could you share the complete address of Viswanath Avenues?",
    "In which state does Mahalakshmi Homes currently reside?",
    "What is the postal code for Udayasri Infra's residence?",
    "Can you provide the address details for Aditya Developers?",
    "What is the current address of Trishala Infrastructure Private Limited?",
    "In which state does S A Developers currently reside?",
    "What is the postal code for Legacy Projects's residence?",
    "Could you please share the complete address of Sahaj Kalyan Gruha Private Limited?",
    "Which state does Varapradha Real Estates Private Limited currently reside in?",
    "Do you have the postal code for Keerthi Estates Private Limited's residence?",
    "What is the address of Orchid Ventures?",
    "What is the postal code for Sree Shakti Developers's residence?",
    "Could you share the complete address of Environ Estates Pvt Ltd?",
    "In which state does Oorjita Builders and Developers Private Limited currently reside?",
    "What is the postal code for Viveks Green Homes Private Limited's residence?",
    "Can you provide the address details for Marwell Homes?",
    "What is the current address of Bliss Homes?",
    "In which state does Bonsai MNP Housing currently reside?",
    "What is the postal code for Quality Builders's residence?",
    "Could you please share the complete address of Jayadev and Others?",
    "Which state does Sri Mytri Constrctions and Deveploers currently reside in?",
    "Do you have the postal code for Abode Developers's residence?",
    "What is the address of M/s TMR Constructions?",
    "What is the postal code for Sreenivasa Constructions's residence?",
    "Could you share the complete address of Bhuvan Grandeur Construction?",
    "In which state does Vision Avenues Private Limited currently reside?",
    "What is the postal code for Virtusa Lifespaces Private Limited's residence?",
    "Can you provide the address details for Rineesh Infra and Developers?",
    "What is the current address of Engreen Custom Homes?",
    "In which state does Sudheer Kumar Gangireddy currently reside?",
    "What is the postal code for Sri Balaji Constructions's residence?",
    "Could you please share the complete address of Shree Vinayaka Lorven Developers?",
    "Which state does Srinivasa Constructions currently reside in?",
    "Do you have the postal code for Shree Maruthi Constructions's residence?",
    "What is the address of Maruthi Builders?",
    "What is the postal code for Smart City Infra Developers's residence?",
    "Could you share the complete address of A R C Projects and Infra?",
    "In which state does Dhanu Constructions Private Limited currently reside?",
    "What is the postal code for Satvika Constructions's residence?",
    "Can you provide the address details for Vipra Ventures?",
    "What is the current address of MPM Hotels Private Limited?",
    "In which state does 1. M/S Vivek's Green Homes Pvt Ltd & 2.M/S Go Green Infra Projects and Others currently reside?",
    "What is the postal code for Emerald Constructions's residence?",
    "Could you please share the complete address of Sri Sai Krupa Constructions?"
]

In [ ]:
new_queries_address = [
    "Who resides at 1-4-1023, Varalaxmi, Gunj Road, Bodhan, Telangana 503185?",
    "Give me details about the resident at Swarnadhama Nagar, Old Bowenpally, Telangana 500011.",
    "What is the current address of Hemanth Kumar in Yellareddyguda, Telangana 500062?",
    "Tell me about the person living in Citizen Colony, Indresham, Sangareddy with the pin code 502319.",
    "In Kukatpally, Telangana 500072, who resides at Hemanas Lake View Heights, Jayanagar?",
    "Who lives in Balaji Nagar, Kukatpally, Telangana 500072, opposite BJP Office?",
    "Can you provide information about the resident of Narapally, Medchal-Malkajgiri, Telangana 500088, in SVS Promoters and Developers?",
    "Tell me about the person residing in Mallial, Jagithyal, Telangana 505452, with the address 7-1-95?",
    "Who resides in Ameerpet, Hyderabad, Telangana 500016, in Shubham Apartments, opposite Bigbazar Lane, near RBI Quarters?",
    "Give me the details for the person living in Dulapally, Medchal-Malkajgiri, Telangana 500100, near Hanuman Temple, in Sanjana's Interlaken?",
    "Who resides in Banjara Hills, Hyderabad, Telangana 500034, in Ashoka Capitol, Road No:2, Opp KBR Park?",
    "Can you share the address details for the person residing at PUPPALAGUDA VILLAGE, Dollar Hills Colony, Dollar Hills, Telangana 500089?",
    "Tell me about the resident of Bachupally, Medchal-Malkajgiri, Telangana 500090, in Hill County, Villa no 235?",
    "What is the current address of Suresh Raju in Miyapur, Hyderabad, Telangana 500049?"
]

In [ ]:
result_str = ""
for qur in old_queries_address:
    result = Q_A_address(qur)
    if result is not None:
        result_str += result

---------------------------------------------------------------------------------------------------
QUERY: Can you provide the current address of Vamshi Constructions?
Similarity Search:- Consider the following information for the address of VAMSI RAM BUILDERS:
- Mandal: Kondapur
- Village/City/Town: KONDAPUR
- Street: KONDAPUR
- Locality: 500084
- State: Ranga Reddy
- District: Serilingampally
- Pin Code:
RESPONSE: The address of Vamshi Constructions is - 500084, Kondapur
---------------------------------------------------------------------------------------------------
QUERY: In which state is the residence of Sri Satya Developers located?
Similarity Search:- Consider the following information for the address of M/S LOHIYA DEVELOPERS PRIVATE LIMITED:
- Mandal: Kismathpur
- Village/City/Town: SRI SATYA SAI N.R.I TOWNSHIP
- Street: SRI SATYA SAI N.R.I TOWNSHIP
- Locality: 500035
- State: Ranga Reddy
- District: Gandipet
- Pin Code:
RESPONSE: The residence of Sri Satya Developers is loc

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-44vZf***************************************mkAs. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
result_str = ""
for qur in new_queries_address:
    result = Q_A_address(qur)
    if result is not None:
        result_str += result

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


---------------------------------------------------------------------------------------------------
QUERY: Who resides at 1-4-1023, Varalaxmi, Gunj Road, Bodhan, Telangana 503185?
Similarity Search:- Consider the following information for the address of UPPALA  VARALAXMI:
- House Number: 1-4-1023
- Building Name: Varalaxmi
- Street Name: Gunj Road
- Locality: Bodhan
- Landmark: Bodhan
- State: Telangana
- District: Nizamabad
- Pin Code: 503185
RESPONSE: UPPALA VARALAXMI lives at 1-4-1023, Varalax
---------------------------------------------------------------------------------------------------
QUERY: Give me details about the resident at Swarnadhama Nagar, Old Bowenpally, Telangana 500011.
Similarity Search:- Consider the following information for the address of KONDAM SRIDHAR REDDY:
- House Number: 8-7-177/50/40,41 FLAT NO 201
- Building Name: KAMALA RESIDENCY
- Street Name: SWARNADHAMA NAGAR
- Locality: OLD BOWENPALLY
- Landmark: VENKATESHWARA TEMPLE
- State: Telangana
- District: M

In [ ]:
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
!ngrok authtoken 2cmUOQLCGYB5wszIPqk71uN5XOG_RjVNchLHzTvkuZ716edy

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask

In [ ]:
from flask import Flask, render_template, jsonify, request

In [ ]:
!pip install flask_cors
from flask_cors import CORS

In [ ]:
!pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.4 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

# Set the port where your Flask app is running
port = 5000

# Start ngrok
public_url = ngrok.connect(port)
print(' * Tunnel URL:', public_url)

 * Tunnel URL: NgrokTunnel: "https://90ba-35-223-57-214.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run
CORS(app)

@app.route("/")
def hello():

  print("hiii")
  return "Hello World!"

@app.route("/api", methods=['POST'])
def receive_data():
  # print("hellow")
  print("im in")
  data = request
  response_data = {'result': 'Data received successfully'}
  print(data,"112")
  # print(request.json)
  print("nooo")
  # data = request.json
  data = request.values.get('Body', '').lower()
  print(data,"199")
  # data = data + "kumar"
  data = Q_A_address(data)
  print(data)
  message = client.messages.create(
  from_='whatsapp:+14155238886',
  body=data,
  to='whatsapp:+918595973928'
)
  return jsonify(data)

if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://90ba-35-223-57-214.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 18:52:22] "GET / HTTP/1.1" 200 -


hiii


...

---------------------------------------------------------------------------------------------------
QUERY: what is the current address of bliss homes?
Similarity Search:- Consider the following information for the address of BLISS HOMES:
- Mandal: 
- Village/City/Town: 
- Street: PADMARAO NAGAR 18
- Locality: PADMARAO NAGAR 18
- State: Telangana
- District: Hyderabad
- Pin Code: 500003
RESPONSE: BLISS HOMES is located in PADMARAO NAGAR 18 in Hyderabad.
BLISS HOMES is located in PADMARAO NAGAR 18 in Hyderabad.
im in
<Request 'http://90ba-35-223-57-214.ngrok-free.app/api' [POST]> 112
nooo
in which state does s a developers currently reside? 199


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 18:56:15] "POST /api HTTP/1.1" 200 -


---------------------------------------------------------------------------------------------------
QUERY: in which state does s a developers currently reside?
Similarity Search:- Consider the following information for the address of ABC DEVELOPERS:
- Mandal: 
- Village/City/Town: SAI NAGAR ROAD
- Street: SAI NAGAR ROAD
- Locality: 507002
- State: Khammam
- District: 
- Pin Code:
RESPONSE: Khammam is the state where ABC DEVELOPERS is located.
Khammam is the state where ABC DEVELOPERS is located.
im in
<Request 'http://90ba-35-223-57-214.ngrok-free.app/api' [POST]> 112
nooo
what is the postal code for rudraksh megacorp private limited's current residence? 199


INFO:werkzeug:127.0.0.1 - - [07/Mar/2024 18:57:12] "POST /api HTTP/1.1" 200 -


---------------------------------------------------------------------------------------------------
QUERY: what is the postal code for rudraksh megacorp private limited's current residence?
Similarity Search:- Consider the following information for the address of Rudraksh MegaCorp Private Limited:
- Mandal: Shamirpet
- Village/City/Town: 
- Street: Shamirpet Village
- Locality: Shamirpet Village
- State: Telangana
- District: Medchal-Malkajgiri
- Pin Code: 500078
RESPONSE: 500078 is the postal code of Rudraksh MegaCorp Private Limited's
500078 is the postal code of Rudraksh MegaCorp Private Limited's


In [ ]:
%pip install --upgrade --quiet  gpt4all > /dev/null

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings

In [ ]:
embeddings = GPT4AllEmbeddings()

In [ ]:
import pickle

# Open the PKL file in binary mode
with open('/content/drive/MyDrive/Address_embedded/index.pkl', 'rb') as file:
    # Load the object from the file
    your_object = pickle.load(file)

# Now, 'your_object' contains the data from the PKL file
